In [9]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from colorama import Fore, Style
from functools import partial
from IPython.display import clear_output

In [10]:
# define a single player
class Player:
    def __init__(self, color, value, strategy=None):
        color_map = {'red':Fore.RED, 'blue':Fore.BLUE}
        self.color = color_map[color]
        self.value = value
        self.points = 0
        self.turns = []
        self.strategy = None

In [11]:
# define force4 game
class Force4:
    def __init__(self, player1:Player, player2:Player):
        self.game = pd.DataFrame(np.zeros((7,7)))
        self.player1 = player1
        self.player2 = player2
        self.current_turn = self.player1
        self.games = 0
        self.turn = 0
        
    def available_moves(self):
        '''Return a (7,) boolean array containing the available moves.
        True -> allowed move.
        False -> not allowed move.
        '''
        return self.game.values[0] == 0
    
    def put_coin(self, value, position):
        '''Put a coin with the passed value (expected +-1.0) at the passed position.
        If the move suggested is not allowed raise an error.'''
        try:
            self.game.values[np.where(self.game.values[:,position] == 0)[0].max(), position] = value
        except:
            print('Move is not allowed!')

    def remove_coin(self, value, position):
        '''Remove a coin from the passed position.'''
        try:
            self.game.values[np.where(self.game.values[:,position] != 0)[0].min(), position] = 0.0
        except:
            print('Cannot remove a coin!')
            
    def has_won(self, value):
        '''Return a boolean referring to whether the player with a given value has won the game with the current grid.'''
        if value < 0:
            vertical = sum([self.game.shift(n, axis=0) for n in range(4)]).fillna(0.0).values.min()
            horizontal = sum([self.game.shift(n, axis=1) for n in range(4)]).fillna(0.0).values.min()
            diag1 = sum([self.game.shift(n, axis=0).shift(n, axis=1) for n in range(4)]).fillna(0.0).values.min()
            diag2 = sum([self.game.shift(n, axis=0).shift(-n, axis=1) for n in range(4)]).fillna(0.0).values.min()
            return min([vertical, horizontal, diag1, diag2]) <= -4.0
        
        if value > 0:
            vertical = sum([self.game.shift(n, axis=0) for n in range(4)]).fillna(0.0).values.max()
            horizontal = sum([self.game.shift(n, axis=1) for n in range(4)]).fillna(0.0).values.max()
            diag1 = sum([self.game.shift(n, axis=0).shift(n, axis=1) for n in range(4)]).fillna(0.0).values.max()
            diag2 = sum([self.game.shift(n, axis=0).shift(-n, axis=1) for n in range(4)]).fillna(0.0).values.max()
            return max([vertical, horizontal, diag1, diag2]) >= 4.0
        
    def play_turn(self, 
                  verbose=True,
                  points=True):
        '''The player passed to the function plays a turn.
        Return a boolean:
        True -> moves available
        False -> player has won or no moves are available.
        '''

        # player put a coin according to its strategy
        position = self.current_turn.strategy()
        self.put_coin(position=position,
                        value=self.current_turn.value)
        # add a turn
        self.turn += 1
            
        # if player1 won
        player_win = self.has_won(value=self.current_turn.value)
        if player_win:
            if verbose:
                print("*"*50)
                if self.current_turn is self.player1:
                    print(self.current_turn.color+"Player 1 win!!!".center(50)+Fore.RESET)
                elif self.current_turn is self.player2:
                    print(self.current_turn.color+"Player 2 win!!!".center(50)+Fore.RESET)
                print("*"*50)
            if points:
                self.current_turn.points += 1
                self.current_turn.turns.append(self.turn)
                self.games += 1
            return False
        # if no moves are available is a draw
        elif True not in self.available_moves():
            if verbose:
                print("*"*50)
                print("It's a draw!!!".center(50))
                print("*"*50)
            self.games += 1
            return False
        
        # next turn the opponent moves
        if self.current_turn is self.player1:
            self.current_turn = self.player2
        else:
            self.current_turn = self.player1
        # else continue playing returning True
        return True
    
    def reset_game(self):
        '''Reset the grid to start a new game'''
        self.game = pd.DataFrame(np.zeros((7,7)))
        self.current_turn = self.player1
        self.turn = 0


    def play_game(self,
                  verbose=True,
                  points=True):
        '''Play a full game until one of the player wins or no more moves are available.'''
        while(self.play_turn(verbose, points)):
            pass
        self.reset_game()
            
    def __str__(self):
        '''Fancy print of the current grid.'''
        grid = pd.DataFrame(self.game.values)
        grid = grid.replace({0:' . ',
                            self.player1.value:self.player1.color+' o '+Fore.RESET,
                            self.player2.value:self.player2.color+' o '+Fore.RESET})
        return ''.join(map(lambda x:''.join(x)+'\n', grid.values.tolist()))

# Strategies

- **random_strategy**: the player put a coin randomly in an available slot.
- **current_best_move_strategy**: the player put a coin to get the highest immediate reward. 
- **input_strategy**: the player takes the move from input.

In [12]:
# define the random strategy
def random_strategy(game:Force4):
    return np.random.choice(np.array([n for n in range(7)])[game.available_moves()])

In [13]:
# the player consider all available moves and returns the one corresponding to the highest reward
def current_best_move_strategy(game:Force4, exponent=4):
    # define a list that will contain a score for each move
    moves = []

    # consider all available moves
    for pos, true_or_false in enumerate(game.available_moves()):
        if true_or_false == True:
            # put a coin that will be removed
            game.put_coin(value=game.current_turn.value,
                        position=pos)

            # consider all horizontal, vertical, and diagonal combination of coins
            vertical = game.current_turn.value * (sum([game.game.shift(n, axis=0) for n in range(4)]).fillna(0.0).values**exponent).sum()
            horizontal = game.current_turn.value * (sum([game.game.shift(n, axis=1) for n in range(4)]).fillna(0.0).values**exponent).sum()
            diag1 = game.current_turn.value * (sum([game.game.shift(n, axis=0).shift(n, axis=1) for n in range(4)]).fillna(0.0).values**exponent).sum()
            diag2 = game.current_turn.value * (sum([game.game.shift(n, axis=0).shift(-n, axis=1) for n in range(4)]).fillna(0.0).values**exponent).sum()
            moves.append( ( pos, sum([vertical, horizontal, diag1, diag2]) ) )

            # remove the coin
            game.remove_coin(value=game.current_turn.value,
                            position=pos)
    
    return max(moves, key=lambda x:x[1])[0]



In [14]:
# input strategy
def input_strategy(game:Force4):
    # print the grid
    print('*'*30)
    print(game)
    print('^'.center(3)*7)
    print('|'.center(3)*7)
    print(''.join([str(n).center(3) for n in range(0, 7)]))
    move = input(game)
    clear_output(wait=False)
    return int(move)

In [15]:
game = Force4(player1=Player(color='red', value=1.0),
              player2=Player(color='blue', value=-1.0))

game.player1.strategy = partial(current_best_move_strategy, game=game)
game.player2.strategy = partial(input_strategy, game=game)

In [16]:
for _ in range(1):
    game.play_game(verbose=False)

In [18]:
game.player1.points

0